In [1]:
import numpy as np
import copy

- __Початкові величини (настрій від типу задач)__

In [2]:
A = [[0.2, 0.3, 0.5],
     [0.2, 0.2, 0.6],
     [0.0, 0.2, 0.8]]

B = [[0.7, 0.2, 0.1],
     [0.3, 0.4, 0.3],
     [0.0, 0.1, 0.9]]

y = [3,1,1,2,2]

time = len(y)

N = len(B)
M = len(B[0])

m = [0.05, 0.2, 0.75]

- __Алгоритм прямого ходу__

In [3]:
def alpha_calculation(m,A,B):
    alpha = [[0.0 for i in range(N)] for t in range(time)]

    for t in range(time):
        for i in range(N):
            if t == 0: 
                alpha[t][i] = m[i]*B[i][y[t]-1]
            else:
                aA = 0
                for j in range(N):
                    aA += alpha[t-1][j]*A[j][i]
                alpha[t][i] = aA*B[i][y[t]-1]

    # for i in range(time):
    #     print(f"Time {i}: {alpha[i]}")

    P_alpha = 0
    for i in range(len(alpha[time-1])):
        P_alpha += alpha[time-1][i]

    return alpha, P_alpha

- __Алгоритм зворотнього ходу__

In [4]:
def beta_calculation(m,A,B):
    beta = [[0.0 for i in range(N)] for t in range(time)]

    for t in range(time-1, -1, -1):
        for i in range(N):
            if t == time-1: 
                beta[t][i] = 1
            else:
                sum = 0
                for j in range(N):
                    sum += beta[t+1][j]*A[i][j]*B[j][y[t+1]-1]
                beta[t][i] = sum

    # for i in range(time-1, -1, -1):
    #     print(f"Time {i}: {beta[i]}")

    P_beta = 0
    for i in range(len(beta[0])):
        P_beta += beta[0][i]*B[i][y[0]-1]*m[i]

    return beta, P_beta

- __Додаткові величини__

In [5]:
def gamma(alpha,beta,P):
    gamma_i = [[0.0 for i in range(N)] for t in range(time)]

    for t in range(time):
        for i in range(N):
            gamma_i[t][i] = alpha[t][i]*beta[t][i]/P

    gamma_ij = [[[0.0 for j in range(N)] for i in range(N)] for t in range(time-1)]

    for t in range(time-1):
        for i in range(N):
            for j in range(N):
                gamma_ij[t][i][j] = alpha[t][i]*A[i][j]*B[j][y[t+1]-1]*beta[t+1][j]/P

    return gamma_i, gamma_ij

- __Переоцінка значень__

In [6]:
def reestimation(m,A,B,gamma_i,gamma_ij):
    for i in range(N):
        m[i] = gamma_i[0][i]

    for i in range(N):
        for j in range(N):
            sum_i, sum_ij = 0, 0
            for t in range(time-1):
                sum_ij += gamma_ij[t][i][j]
                sum_i += gamma_i[t][i]
            A[i][j] = sum_ij/sum_i

    for j in range(N):
        for k in range(M):
            sum_up, sum_down = 0, 0
            for t in range(time):
                if y[t] == k+1: sum_up += gamma_i[t][j]
                sum_down += gamma_i[t][j]
            B[j][k] = sum_up/sum_down

    return m, A, B

- __Задача І (оцінювання)__

In [7]:
P = [[] for i in range(3*3*3*3*3)]

k = 0
for i_1 in range(3):
    for i_2 in range(3):
        for i_3 in range(3):
            for i_4 in range(3):
                for i_5 in range(3):
                    y = [i_1+1, i_2+1, i_3+1, i_4+1, i_5+1]
                    time = len(y)

                    alpha, P_alpha = alpha_calculation(m,A,B)
                    P[k].append(copy.deepcopy(P_alpha))
                    P[k].append(copy.deepcopy(y))
                    k += 1

candidates = []
for i in range(len(P)):
    candidates.append(P[i][0])
print(max(candidates), P[np.argmax(candidates)][1])

# print(candidates[200:243])

# for i in range(len(P)):
#     if P[i][1] == [3,1,1,2,2]:
#         print(P[i][0])

p = 0
for i in range(len(P)):
    if P[i][1][1] == 1 and P[i][1][2] == 1:
        p += P[i][0]

0.2555899124000002 [3, 3, 3, 3, 3]


In [8]:
y = [3,1,1,2,2]

alpha, P_alpha = alpha_calculation(m,A,B)
beta, P_beta = beta_calculation(m,A,B)

for i in range(len(alpha)):
    print(alpha[i])

for i in range(len(beta)):
    print(beta[i])

print(P_alpha, P_beta)

[0.005000000000000001, 0.06, 0.675]
[0.0091, 0.044550000000000006, 0.0]
[0.007511, 0.0034920000000000003, 0.0]
[0.0004401200000000001, 0.0011806800000000001, 0.00058507]
[6.483200000000002e-05, 0.00019407440000000005, 0.00013965240000000004]
[0.0018068000000000003, 0.0015888800000000004, 0.0004358400000000001]
[0.008236000000000002, 0.007264000000000001, 0.0019440000000000008]
[0.038000000000000006, 0.03240000000000001, 0.02720000000000001]
[0.21000000000000002, 0.18000000000000002, 0.16000000000000003]
[1, 1, 1]
0.0003985588000000001 0.0003985588000000001


- __Задача ІІ (навчання)__

In [9]:
A = [[0.2, 0.3, 0.5],
     [0.2, 0.2, 0.6],
     [0.0, 0.2, 0.8]]

B = [[0.7, 0.2, 0.1],
     [0.3, 0.4, 0.3],
     [0.0, 0.1, 0.9]]

y = [3,1,1,2,2]

N = len(B)
M = len(B[0])

m = [0.05, 0.2, 0.75]

time = len(y)

In [10]:
A = [[0.34, 0.33, 0.33],
     [0.33, 0.34, 0.33],
     [0.33, 0.33, 0.34]]

B = [[0.34, 0.33, 0.33],
     [0.33, 0.34, 0.33],
     [0.33, 0.33, 0.34]]

y = [3,1,1,2,2]

N = len(B)
M = len(B[0])

m = [0.33, 0.34, 0.33]

time = len(y)

In [11]:
number_of_iterations = 23
P_alpha = 0

for i in range(number_of_iterations):
    delta = P_alpha

    alpha, P_alpha = alpha_calculation(m,A,B)
    beta, P_beta = beta_calculation(m,A,B)
    gamma_i, gamma_ij = gamma(alpha,beta,P_alpha)
    m,A,B = reestimation(m,A,B,gamma_i,gamma_ij)

    delta = P_alpha - delta
    print(f"k={i} & {round(P_alpha,6)} & {round(delta,6)} \\\\")

print("\nP =", P_alpha)

print("\nm:\n", m, "\n\nA:")
for i in range(len(A)):
    print(A[i])
print("\nB:")
for i in range(N):
    print(B[i])

k=0 & 0.004115 & 0.004115 \\
k=1 & 0.005121 & 0.001007 \\
k=2 & 0.005126 & 4e-06 \\
k=3 & 0.005142 & 1.6e-05 \\
k=4 & 0.005206 & 6.4e-05 \\
k=5 & 0.00545 & 0.000244 \\
k=6 & 0.006319 & 0.000869 \\
k=7 & 0.008941 & 0.002622 \\
k=8 & 0.015249 & 0.006308 \\
k=9 & 0.029011 & 0.013762 \\
k=10 & 0.048554 & 0.019543 \\
k=11 & 0.060727 & 0.012173 \\
k=12 & 0.063046 & 0.002319 \\
k=13 & 0.063939 & 0.000892 \\
k=14 & 0.06606 & 0.002122 \\
k=15 & 0.0713 & 0.00524 \\
k=16 & 0.084096 & 0.012796 \\
k=17 & 0.112732 & 0.028636 \\
k=18 & 0.161779 & 0.049047 \\
k=19 & 0.212718 & 0.050939 \\
k=20 & 0.238689 & 0.025971 \\
k=21 & 0.246017 & 0.007329 \\
k=22 & 0.247957 & 0.00194 \\

P = 0.24795745827095714

m:
 [0.0, 0.0, 1.0000000000000002] 

A:
[0.47207029085097546, 0.5279297091490245, 0.0]
[4.0930862170117625e-10, 0.9999999995906914, 0.0]
[0.9999999999494231, 5.0576734410649967e-11, 0.0]

B:
[0.9996875062150524, 0.0003124937849476858, 0.0]
[0.05052738127029629, 0.9494726187297038, 0.0]
[0.0, 0.0, 1.0]


- __Задача ІІІ (декодування)__

In [12]:
A = [[0.2, 0.3, 0.5],
     [0.2, 0.2, 0.6],
     [0.0, 0.2, 0.8]]

B = [[0.7, 0.2, 0.1],
     [0.3, 0.4, 0.3],
     [0.0, 0.1, 0.9]]

N = len(B)
M = len(B[0])

m = [0.05, 0.2, 0.75]

# y = [1,3,2,1,3]
# y = [3,1,1,2,2]
y = [3,3,3,3,3]

time = len(y)

- __Алгоритм Вітербі__

In [13]:
def viterbiL(y,m,A,B):
    time = len(y)
    delta = [[0.0 for i in range(len(B))] for t in range(time)]
    psi = [[0 for i in range(len(B))] for t in range(time)]

    for t in range(time):
        for i in range(len(B)):
            if t == 0: 
                delta[t][i] = m[i]*B[i][y[t]]
            else:
                dA = []
                for j in range(len(B)):
                    dA.append(delta[t-1][j]*A[j][i]*B[i][y[t]])
                delta[t][i] = max(dA)
                psi[t][i] = np.argmax(dA)

    x = []
    delta_hat = max(delta[time-1])
    x.append(np.argmax(delta[time-1]))

    for t in range(time-2, -1, -1):
        x.insert(0, psi[t+1][x[0]])

    return x

In [14]:
def viterbi_SF(y,A,B,Pi=None):
    """
    Return the MAP estimate of state trajectory of Hidden Markov Model.

    Parameters
    ----------
    y : array (T,)
        Observation state sequence. int dtype.
    A : array (K, K)
        State transition matrix. See HiddenMarkovModel.state_transition  for
        details.
    B : array (K, M)
        Emission matrix. See HiddenMarkovModel.emission for details.
    Pi: optional, (K,)
        Initial state probabilities: Pi[i] is the probability x[0] == i. If
        None, uniform initial distribution is assumed (Pi[:] == 1/K).

    Returns
    -------
    x : array (T,)
        Maximum a posteriori probability estimate of hidden state trajectory,
        conditioned on observation sequence y under the model parameters A, B,
        Pi.
    T1: array (K, T)
        the probability of the most likely path so far
    T2: array (K, T)
        the x_j-1 of the most likely path so far
    """
    # Cardinality of the state space
    K = A.shape[0]
    # Initialize the priors with default (uniform dist) if not given by caller
    Pi = Pi if Pi is not None else np.full(K, 1 / K)
    T = len(y)
    T1 = np.empty((K, T), 'd')
    T2 = np.empty((K, T), 'B')

    # Initilaize the tracking tables from first observation
    T1[:, 0] = Pi * B[:, y[0]]
    T2[:, 0] = 0

    # Iterate throught the observations updating the tracking tables
    for i in range(1, T):
        T1[:, i] = np.max(T1[:, i - 1] * A.T * B[np.newaxis, :, y[i]-1].T, 1)
        T2[:, i] = np.argmax(T1[:, i - 1] * A.T, 1)

    # Build the output, optimal model trajectory
    x = np.empty(T, 'B')
    x[-1] = np.argmax(T1[:, T - 1])
    for i in reversed(range(1, T)):
        x[i - 1] = T2[x[i], i]

    return x

In [15]:
def viterbi(y,m,A,B):
    delta = [[0.0 for i in range(N)] for t in range(time)]
    psi = [[0 for i in range(N)] for t in range(time)]

    for t in range(time):
        for i in range(N):
            if t == 0: 
                delta[t][i] = m[i]*B[i][y[t]-1]
            else:
                dA = []
                for j in range(N):
                    dA.append(delta[t-1][j]*A[j][i]*B[i][y[t]-1])
                delta[t][i] = max(dA)
                psi[t][i] = np.argmax(dA) + 1

    # print(delta)
    # print(psi)

    x = []
    delta_hat = max(delta[time-1])
    x.append(np.argmax(delta[time-1])+1)

    for t in range(time-2, -1, -1):
        x.insert(0, psi[t+1][x[0]-1])

    return x

In [16]:
def log_viterbi(y,m,A,B):
    time = len(y)

    delta = [[0.0 for i in range(len(B))] for t in range(time)]
    psi = [[0 for i in range(len(B))] for t in range(time)]

    for t in range(time):
        for i in range(len(B)):
            if t == 0:
                # delta[t][i] = m[i]*B[i][y[t]-1]
                delta[t][i] = np.log(m[i]) + np.log(B[i][y[t]-1])
            else:
                dA = []
                for j in range(len(B)):
                    # dA.append(delta[t-1][j]*A[j][i]*B[i][y[t]-1])
                    dA.append(delta[t-1][j] + np.log(A[j][i]) + np.log(B[i][y[t]-1]))
                delta[t][i] = max(dA)
                psi[t][i] = np.argmax(dA) + 1
    
    x = []
    delta_hat = max(delta[time-1])
    x.append(np.argmax(delta[time-1])+1)

    for t in range(time-2, -1, -1):
        x.insert(0, psi[t+1][x[0]-1])

    return x

In [17]:
def log_viterbi_nilsson(y,m,A,B):
    delta = [[0.0 for i in range(N)] for t in range(time)]
    psi = [[0 for i in range(N)] for t in range(time)]

    for t in range(time):
        for i in range(N):
            if t == 0: 
                delta[t][i] = np.log(m[i]) + np.log(B[i][y[t]-1])
            else:
                dA = []
                for j in range(N):
                    dA.append(delta[t-1][j] + np.log(A[j][i]))
                delta[t][i] = np.log(B[i][y[t]-1]) + max(dA)
                psi[t][i] = np.argmax(dA) + 1

    # print(delta)
    # print(psi)

    x = []
    delta_hat = max(delta[time-1])
    x.append(np.argmax(delta[time-1])+1)

    for t in range(time-2, -1, -1):
        x.insert(0, psi[t+1][x[0]-1])

    return x

In [19]:
print(f"\nOrigin: {viterbi(y,m,A,B)}")
print(f"\nOrigin: {viterbiL([y[i]-1 for i in range(len(y))],m,A,B)}")
print(f"\nLog   : {log_viterbi(y,m,A,B)}")

x_1 = viterbi(y,m,A,B)
x_2 = log_viterbi(y,m,A,B)

k = 0
for i in range(len(x_1)):
    if x_1[i] != x_2[i]: 
        k += 1

print("\nMisplaces =", k)


Origin: [3, 3, 3, 3, 3]

Origin: [2, 2, 2, 2, 2]

Log   : [3, 3, 3, 3, 3]

Misplaces = 0


/tmp/ipykernel_38647/3884816887.py:16: RuntimeWarning: divide by zero encountered in log
  dA.append(delta[t-1][j] + np.log(A[j][i]) + np.log(B[i][y[t]-1]))
